In [2]:
# pip install langchain_community

In [3]:
# pip install langchain_openai

In [4]:
# pip install pysqlite3

In [5]:
# pip install Chroma

In [7]:
# pip install pymupdf

In [8]:
# pip install langchain-anthropic

In [9]:
import os
import sys

# Import classes from modules
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic

__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [10]:
# Generate API KEY from Claude and OpenAI website and define as a variable.
os.environ["ANTHROPIC_API_KEY"] = "YOUR_API_KEYS"
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEYS"

In [11]:
# Define the path to your local PDF file
local_path = "ข้อมูลยา 50 ชนิด.pdf"

# Load and split the PDF file
if local_path:
    loader = PyMuPDFLoader(file_path=local_path)
    data = loader.load_and_split()
else:
    print("Upload a PDF file")
    sys.exit()

In [12]:
# data

In [13]:
text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [14]:
# texts

In [15]:
# This part is used for embedding the docs and storing them into Vector DB and initializing the retriever.
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

In [ ]:
custom_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    You are an AI thai language model assistant.
    You are an expert at answering questions about medicine.
    Answer the question based ONLY on the following context.


    {context}


    Original question: {question}""",
)

In [16]:
# Create the language model
llm1 = ChatAnthropic(model="claude-3-sonnet-20240229", max_tokens=1024)

chain = ConversationalRetrievalChain.from_llm(
    llm=llm1,
    retriever=docsearch.as_retriever(),
    combine_docs_chain_kwargs={"prompt": custom_template}
)

In [17]:
chat_history = []
query = None  # Initialize query to avoid potential reference error

while True:
    if not query:
        query = input("User: ")
    if query in ['quit', 'q', 'exit']:
        break
    result = chain.invoke({"question": query, "chat_history": chat_history})
    print("Chatbot:", result['answer'])

    chat_history.append((query, result['answer']))
    query = None

User: q
